In [1]:
"""
Update Parameters Here
"""
COLLECTION = "MekaVerse"
CONTRACT = "0x9A534628B4062E123cE7Ee2222ec20B86e16Ca8F"
YEAR = "2021"
MONTH = "10"
DAY = "13"
HOUR = "15"
MINUTE = "00"

SLEEP = 5
LIMIT = 200
MAX_OFFSET = 10000

OPENSEA_API_KEY = "YOUR_API_KEY"
METHOD = "raritytools"
TOKEN_COL = "TOKEN_ID"

In [2]:
"""
@author: mdigi14
"""

import pandas as pd
import matplotlib.pyplot as plt
import datetime
import time
import requests

REVEAL_TIME = "{}-{}-{}T{}:{}:00".format(YEAR, MONTH, DAY, HOUR, MINUTE)
DATETIME_REVEAL_TIME = datetime.datetime.strptime(REVEAL_TIME, "%Y-%m-%dT%H:%M:%S")
ETHER_UNITS = 1e18


"""
Plot params
"""
plt.rcParams.update({"figure.facecolor": "white", "savefig.facecolor": "white"})

"""
Helper Functions
"""


def getOpenseaEvents(contract, offset=0, occurred_before=REVEAL_TIME):
    print(offset)
    url = "https://api.opensea.io/api/v1/events"

    querystring = {
        "asset_contract_address": contract,
        "only_opensea": "false",
        "offset": str(offset),
        "limit": LIMIT,
        "event_type": "bid_entered",
        "occurred_before": REVEAL_TIME,
    }

    headers = {"Accept": "application/json", "X-API-KEY": OPENSEA_API_KEY}

    response = requests.request("GET", url, headers=headers, params=querystring)
    return response.json()

2021-10-13T15:00:00
2021-10-13 15:00:00


In [3]:
RARITY_CSV = "../metadata/rarity_data/{}_{}.csv".format(COLLECTION, METHOD)
RARITY_DB = pd.read_csv(RARITY_CSV)

bids = []
events = []

offset = 0
while offset < MAX_OFFSET:
    new_events = getOpenseaEvents(CONTRACT, offset=offset)

    for event in new_events["asset_events"]:
        events.append(event)
    offset += LIMIT

    if len(new_events["asset_events"]) < LIMIT:
        break

print("total bids ", len(events))
for event in events:
    bid_time = datetime.datetime.strptime(event["created_date"], "%Y-%m-%dT%H:%M:%S.%f")
    if bid_time < DATETIME_REVEAL_TIME:

        try:
            tokenId = int(event["asset"][TOKEN_COL])
            bid = dict()
            bid[TOKEN_COL] = tokenId
            bid["USER"] = event["from_account"]["address"]
            bid["OFFER"] = float(event["bid_amount"]) / ETHER_UNITS
            bid["DATE"] = event["created_date"]
            bid["RANK"] = int(RARITY_DB[RARITY_DB[TOKEN_COL] == tokenId]["Rank"])
            bids.append(bid)

        except:
            continue


bidding_df = pd.DataFrame(bids)
bidding_df.to_csv("pre-reveal_bids/{}_pre-reveal_bids.csv".format(COLLECTION))


ax = bidding_df.plot.scatter(
    x="TOKEN_ID",
    y="RANK",
    alpha=0.25,
    title="{} - {}".format(COLLECTION, "Pre-reveal Bids"),
    figsize=(14, 7),
)
ax.set_xlabel("Token ID")
ax.set_ylabel("Rarity Rank")

0


KeyError: 'asset_events'